## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cherskiy,68.7500,161.3000,-9.76,100,100,5.21,RU,2021-10-20 16:53:15
1,1,Ushuaia,-54.8000,-68.3000,49.62,53,75,23.02,AR,2021-10-20 16:53:16
2,2,Konakovo,56.7000,36.7667,33.01,94,100,15.12,RU,2021-10-20 16:53:16
3,3,Skhisma,35.2667,25.7167,66.83,67,5,17.52,GR,2021-10-20 16:53:16
4,4,Bethel,41.3712,-73.4140,73.94,61,61,1.01,US,2021-10-20 16:53:17


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Atuona,-9.8000,-139.0333,76.35,83,28,12.62,PF,2021-10-20 16:53:18
6,6,Dawei,14.0833,98.2000,76.24,91,100,1.70,MM,2021-10-20 16:53:18
7,7,Puerto Ayora,-0.7393,-90.3518,76.96,80,94,11.12,EC,2021-10-20 16:53:19
14,14,Butaritari,3.0707,172.7902,81.23,72,10,6.96,KI,2021-10-20 16:53:22
17,17,Harper,4.3750,-7.7169,80.69,80,91,9.86,LR,2021-10-20 16:46:55
18,18,Port-Gentil,-0.7193,8.7815,80.56,83,75,9.22,GA,2021-10-20 16:53:23
23,23,Tamale,9.4008,-0.8393,85.51,63,97,5.41,GH,2021-10-20 16:52:05
25,25,Qiongshan,20.0058,110.3542,75.18,100,0,0.00,CN,2021-10-20 16:53:26
26,26,Hermanus,-34.4187,19.2345,75.90,67,90,12.03,ZA,2021-10-20 16:53:27
27,27,Victoria,22.2855,114.1577,80.58,86,88,1.01,HK,2021-10-20 16:53:27


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       205
City          205
Lat           205
Lng           205
Max Temp      205
Humidity      205
Cloudiness    205
Wind Speed    205
Country       204
Date          205
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)

/Users/jennyliang/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
preferred_cities_df.count()

City_ID       204
City          204
Lat           204
Lng           204
Max Temp      204
Humidity      204
Cloudiness    204
Wind Speed    204
Country       204
Date          204
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,76.35,-9.8000,-139.0333,
6,Dawei,MM,76.24,14.0833,98.2000,
7,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
14,Butaritari,KI,81.23,3.0707,172.7902,
17,Harper,LR,80.69,4.3750,-7.7169,
18,Port-Gentil,GA,80.56,-0.7193,8.7815,
23,Tamale,GH,85.51,9.4008,-0.8393,
25,Qiongshan,CN,75.18,20.0058,110.3542,
26,Hermanus,ZA,75.90,-34.4187,19.2345,
27,Victoria,HK,80.58,22.2855,114.1577,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,76.35,-9.8000,-139.0333,Villa Enata
6,Dawei,MM,76.24,14.0833,98.2000,Hotel Dawei
7,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Butaritari,KI,81.23,3.0707,172.7902,Isles Sunset Lodge
17,Harper,LR,80.69,4.3750,-7.7169,Screensaver
...,...,...,...,...,...,...
691,Manokwari,ID,78.46,-0.8667,134.0833,Swiss Belhotel Manokwari
693,Kudahuvadhoo,MV,83.07,2.6708,72.8944,Niyama Private Islands Maldives
694,Soure,BR,85.51,-0.7167,-48.5233,Casarão da Amazônia
695,Bima,ID,78.93,-8.4667,118.7167,Lambitu Hotel


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [12]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))